# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://edwarddonner.com")
print(ed.get_contents())
ed.links

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers a

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/01/23/ll

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Qwen/Qwen3-235B-A22B
Updated
5 days ago
•
30.5k
•
683
deepseek-ai/DeepSeek-Prover-V2-671B
Updated
5 days ago
•
2.03k
•
655
Qwen/Qwen3-30B-A3B
Updated
6 days ago
•
67.6k
•
446
nari-labs/Dia-1.6B
Updated
9 days ago
•
113k
•
1.83k
Qwen/Qwen3-32B
Updated
6 days ago
•
87.4k
•
269
Browse 1M+ models
Spaces
Running
5.94k
5.94k
DeepSite
🐳
Generate any application with DeepSeek
Ru

In [23]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information. I want you to return the result in Italian."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nQwen/Qwen3-235B-A22B\nUpdated\n5 days ago\n•\n30.5k\n•\n683\ndeepseek-ai/DeepSeek-Prover-V2-671B\nUpdated\n5 days ago\n•\n2.03k\n•\n655\nQwen/Qwen3-30B-A3B\nUpdated\n6 days ago\n•\n67.6k\n•\n446\nnari-labs/Dia-1.6B\nUpdated\n9 days ago\n•\n113k\n•\n1.83k\nQwen/Qwen3-32B\nUpdated\n6 days ago\n•\n87.4k\n•\n269\nBrowse 1M+ models\nSpaces\nRunning\n5.94k\n5.94k\nDeepSite\n🐳\nGenerate any applicati

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}


# Hugging Face Brochure

## About Us
Hugging Face is a pioneering platform at the forefront of artificial intelligence, known for fostering collaboration within the comprehensive machine learning community. We empower users to connect, create, and innovate through access to cutting-edge models, datasets, and applications. Our mission is simple yet profound: to build the future of AI together.

## Community
At Hugging Face, we pride ourselves on being a vibrant community where industry leaders, researchers, and developers come together to share knowledge and resources. With over **1 million models** and **250k datasets**, our platform is the go-to destination for AI enthusiasts and professionals alike. More than **50,000 organizations** have leveraged our tools, including giants like Amazon, Google, Microsoft, and Intel.

### Collaboration Made Easy
- **Models:** Access a vast collection of models across various fields.
- **Datasets:** Discover datasets tailored for any machine learning task.
- **Spaces:** Create and host applications seamlessly in a collaborative environment.

## Company Culture
Hugging Face champions innovation, transparency, and collaboration. Our open-source ethos allows us to build a solid foundation for machine learning tooling with the community. With an emphasis on continuous learning, our team grows alongside the rapidly evolving AI landscape.

### Join Us
We are always on the lookout for passionate individuals ready to contribute to the AI revolution. Careers at Hugging Face offer an opportunity to work with some of the brightest minds in the industry, while also giving you the freedom to explore and grow in your career.

**Current Open Positions:**
- Software Engineer
- Data Scientist
- Community Manager
- AI Researcher

Explore career opportunities at [Hugging Face Careers](https://huggingface.co/jobs).

## Our Offerings
Hugging Face provides solutions for both individuals and enterprises, ensuring a versatile experience catered to various needs.

- **For Individuals:** 
  - Free access to models and datasets.
  - Community support and resources to build your ML profile.

- **For Enterprises:**
  - Dedicated tools and support to enhance AI development.
  - Affordable pricing starting at **$20/user/month** with enterprise-grade security.

### Get Started
Ready to dive into the world of AI? 
- **Sign Up** today to unlock a wealth of resources and start your journey with us!

## Connect With Us
Stay updated and connected. Join the Hugging Face community on:
- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://linkedin.com/company/huggingface)
- [Discord](https://discord.gg/huggingface)

Together, let’s shape the future of AI. Welcome to Hugging Face! 🌟

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/#about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'social media page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Hugging Face: The AI Community Building the Future

## About Us
Hugging Face is an innovative platform designed for the machine learning community, enabling collaboration on a diverse array of models, datasets, and applications. With over 1 million models available and extensive support for various modalities—text, image, video, audio, and 3D—Hugging Face is at the forefront of AI development, fostering a collaborative environment that accelerates machine learning projects globally.

## Our Offerings
- **Models:** Explore and utilize a myriad of AI models spanning numerous applications.
- **Datasets:** Access a wide range of datasets for various machine learning tasks.
- **Spaces:** Collaborate on applications and host unlimited public models, datasets, and applications.
- **Enterprise Solutions:** Tailored services for teams, featuring enterprise-grade security, access controls, and dedicated support.

## Trusted by Leading Organizations
Hugging Face is trusted by over 50,000 organizations, including prestigious companies such as:
- **Meta**
- **Amazon**
- **Google**
- **Microsoft**
- **Intel**
- **Grammarly**

## Company Culture
At Hugging Face, we believe in the power of community and openness. Our culture is built around collaboration, innovation, and shared knowledge. We strive to create an inclusive environment where every team member can contribute, learn, and grow. By valuing transparency and open-source principles, we empower our users and staff to push the boundaries of what is possible in AI.

## Careers at Hugging Face
Join us in shaping the future of AI! We are always on the lookout for talented and passionate individuals to join our team. Whether you are an engineer, researcher, or someone with a flair for creativity, there are countless opportunities to make an impact. Check out our [careers page](#) for current openings and become part of a community that is transforming the way AI is developed and utilized.

## Connect with Us
- **Social Media:**
  - [Twitter](#)
  - [LinkedIn](#)
  - [GitHub](#)
  - [Discord](#)

Explore Hugging Face today and discover how we can collaborate on the future of AI together!



In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community discussion page', 'url': 'https://discuss.huggingface.co'}]}


# Brochure di Hugging Face

## Benvenuti nel Futuro dell'AI!

### Chi Siamo?
➡️ **Hugging Face**: La comunità di AI che abbraccia il futuro! Siamo qui per creare, collaborare e farci un sacco di nuove idee che brillano come un po' di codice ben fatto.

### Cosa Facciamo?
- 🎉 **Modelli di AI**: Con oltre **1 milione di modelli** pronti per essere esplorati, potresti dire che abbiamo modelli anche per l’imprevisto! (Cozze, chi l'avrebbe mai detto?)
- 📊 **Spazi e Applicazioni**: Costruisci qualsiasi applicazione tu possa immaginare, o anche quelle che non puoi! Con DeepSeek, chiunque può diventare un creatore. Solo attenzione a non generare un clone di te stesso!
- 📚 **Dataset**: Oltre **250.000 dataset** a portata di click. Dai un occhio ai dati e scoprire quanti di loro amano ballare come i dati di Excel di lunedì mattina.

### Clienti Sorridenti
👉 Siamo in ottima compagnia! Più di **50.000 organizzazioni** fiducioseutz: da Google e Microsoft fino a piccole startup che non sanno nemmeno che colore usare nei loro loghi. 

### Cultura Aziendale
🎈 In Hugging Face, crediamo nel lavoro chimico e nella collaborazione – anche se a volte sembra più come un laboratorio di follia! Mettiamo il nostro cuore (e molte risate) nel codice.

### Carrriere
🚀 Vuoi unirti a noi? Stiamo cercano *alchimisti del codice*, guru dell'AI, e chiunque possa fare caffè (sì, caffè è un requisito essenziale!) - Vieni a dare un abbraccio al futuro dell'AI!

### I Nostri "Super-Poteri"
- **Open Source**: Contribuiamo a rendere la tecnologia dell'AI accessibile.
- **Supporto Impeccabile**: Se hai problemi, abbiamo una squadra che si preoccupa per te - più di quanto le tue piante d’appartamento facciano (almeno spero).
  
### Prezzi
💵 Dai un'occhiata alle nostre opzioni di abbonamento: partiamo da soli **$0.60/ora** per l'uso del GPU. Quando si dice che puoi costruire un impero a buon mercato…

### Contattaci!
📬 Ci piacerebbe sentire la tua storia! Visita il nostro sito web o scrivici su Twitter, possiamo mandarti anche un *gattino AI* (figurato, ovviamente!).

---

**Hugging Face**: Se pensi che AI e divertimento non possano coesistere, è tempo di un abbraccio! 🤗

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>